In [290]:
import numpy as np
import sys
import random
from phe import paillier

sys.path.append("../")

In [291]:
from federated.data.mitbih_data_preprocessing import load_data, create_unbalanced_data

In [292]:
train_client_data, test_client_data = load_data(normalized=True, data_selector=create_unbalanced_data)

In [293]:
def get_data_for_clients(source):
    data = []
    for client in range(1, 6):
        client_data = source[f"client_{client}"]
        X, y = client_data[0], client_data[1]
        client_data = [(X[i].reshape(-1, 1), y[i].reshape(-1, 1)) for i in range(len(X))]
        data.append(client_data)
    return data

In [322]:
class CrossEntropyCost:

    @staticmethod
    def fn(a, y):
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    
    @staticmethod
    def delta(z, a, y):
        return (a-y)
    
class Network:
    
    def __init__(self, sizes, weights, biases, cost=CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = biases
        self.weights = weights
        self.cost = cost
    
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta):
            n = len(training_data)
            for j in range(epochs):
                random.shuffle(training_data)
                mini_batches = [
                    training_data[k:k+mini_batch_size]
                    for k in range(0, n, mini_batch_size)]
                for mini_batch in mini_batches:
                    self.update_mini_batch(
                        mini_batch, eta, n)
        
    def update_mini_batch(self, mini_batch, eta, n):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
    
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def accuracy(self, data):
        results = [(np.argmax(self.feedforward(x)), y)
                       for (x, y) in data]
        return sum(int(y[x] == 1.0) for (x, y) in results)
    
    def total_cost(self, data):
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            cost += self.cost.fn(a, y)/len(data)
        return cost

def sigmoid(z):
    # return z
    return 1.0/(1.0+np.exp(-1*z))

def sigmoid_prime(z):
    # return 1
    return sigmoid(z)*(1-sigmoid(z))

In [323]:
class Client:
    def __init__(self, name, data, model):
        self.name = name
        self.data = data
        self.model = model
    
    def train(self):
        self.model.SGD(self.data, 15, 32, 0.02)

In [324]:
class Server:
    def __init__(self, model):
        self.model = model
    
    def aggregate(self, updates):
        weight_updates = [update.weights for update in updates]
        bias_updates = [update.biases for update in updates]

        self.model.weights = np.sum(weight_updates, axis=0) / len(weight_updates)
        self.model.biases = np.sum(bias_updates, axis=0) / len(bias_updates)

In [325]:
sizes = [186, 5]
weights = [np.random.randn(y, x)/np.sqrt(x) for x, y in zip(sizes[:-1], sizes[1:])]
biases = [np.random.randn(y, 1) for y in sizes[1:]]

In [310]:
keypair = paillier.generate_paillier_keypair(n_length=1024)
pubkey, privkey = keypair

In [311]:
def encrypt(matrix):
    matrix_list = []
    for i in range(len(matrix)):
        arr1 = []
        for j in range(matrix[i].shape[0]):
            arr2 = []
            for k in range(matrix[i].shape[1]):
                arr2.append(pubkey.encrypt(matrix[i][j][k]))
        
            arr2 = np.array(arr2)
            arr1.append(arr2)

        arr1 = np.array(arr1)
        matrix_list.append(arr1)
    
    return matrix_list

In [312]:
weights_encrypted = encrypt(weights)
biases_encrypted = encrypt(biases)

In [326]:
model = Network(sizes, weights_encrypted, biases_encrypted)
server = Server(model)
data = get_data_for_clients(train_client_data)

In [327]:
for round_number in range(1):
    clients = [Client(f"client_{i+1}", data[i], server.model) for i in range(1)]
    
    for client in clients:
        client.train()
    
    updates = [client.model for client in clients]

    server.aggregate(updates)

    print(f"Round {round_number} completed :)")

ValueError: Integer needs to be within +/- 40253201466422529018124117074196576056941202661166782917965324518540723482236231320902639116951941804866436374937630174853545062793432128182136376506857951400129720850904548148524833991041285970686564493794519959527234164396434875691647244215476593108032837423664762343455007945305504394268256558694622430158 but got -3162535207926728411757739792483082922649322652564720599326583719178599055749927295186880995334955070237298091629950186115465761521653023076047084374074039217523709754321349396319795433470206505331871369976136641696281366928510823415907864932018118067825157133005605774255209603362976602065411367322593847864182933454585856